In [ ]:
include("./Transit.jl")
include("../bls.jl")
using Transit
using PyPlot
using DataFrames
using LsqFit

In [ ]:
# get the radial velocity data

transit_data = readdlm("../mysteryplanet2.txt");
#t = transit_data[1];
#flux = transit_data[2];
t = transit_data[:,1]
flux = transit_data[:,2]

n = length(t)
dt = t[2] - t[1]
t_tot = maximum(t) - minimum(t)
t_min = 0.5/24.
t_max = 10./24.
qmi = t_min/t_tot
qma = t_max/t_tot
fmin = 1.0/t_tot
fmax = 1.0/0.5
nf = 1000
df = (fmax - fmin)/nf
nb = 1000;

In [ ]:
p, bper, bpow, depth, qtran, in1, in2, f0 = bls(n,t,flux,nf,fmin,df,nb,qmi,qma);

In [ ]:
t_fold = t % bper

plot(t_fold, flux, ".")  

In [ ]:
function model(t, arr)
    rp, t0, dur, b = arr
    return transit(t, rp, 1, t0, dur, b)
end
fit = curve_fit(model, t_fold, flux, [0.1, 0.5, 0.2, 0.0]);

In [ ]:
plot(t_fold, flux, ".")
plot(t_fold, model(t_fold, fit.param), "r.")
G = 6.6741e-11
T = fit.param[3]*60*60*24
b = fit.param[4]
P = bper*60*60*24
dens = (3/(pi^2))*(P/(G*(T^3)))*((1-b^2)^1.5)
println(fit.param)
println(dens)

Looks like...

Rp/Rstar = 0.00951

time of first contact = 0.510

duration = 0.185

b ~ 0

dens = 0.427 g/cm^3

But this all changes a lot depending on initial guesses - and I can see why. With so much noise, there's very little to constrain the exact location and depth of the transit. Really easy to get a small xi squared over a wide range of parameters. 

In [ ]:
function quadratic(mu, ld_args)
    a1, a2 = ld_args
    A = 1./(1 - a1/2. - a2/3.)
    return A*(1 - a1*(1-mu) - a2*((1-mu).^2))
end

function nonlinear(mu, ld_args)
    a1, a2, a3, a4 = ld_args
    A = 1./(1 - a1/3. - a2/2. - 3*a3/5. - 2*a4/3.)
    return A*(1 - a1*(1-mu.^0.5) - a2*(1-mu) - a3*(1-mu.^1.5) - a4*(1-mu.^2))
end

m = linspace(0,1,1000)
plot(m, quadratic(m, (0.0, 0.5)))

In [ ]:
plot(t, ld_transit(t, 0.1, 2.5, 25, 0.5, quadratic, (0.0,0.5)))
plot(t, ld_transit(t, 0.1, 2.5, 25, 0.5, quadratic, (0.5,0.5)))
plot(t, ld_transit(t, 0.1, 2.5, 25, 0.5, nonlinear, (0.5, 0.1, 0.1, -0.1)))
plot(t, ld_transit(t, 0.1, 2.5, 25, 0.5, quadratic, (0.0,0.0)))